In [ ]:
import xlwings as xw
import pandas as pd
from nsepython import *
from datetime import datetime, timedelta

In [12]:
def gethighofadate(symbol, dt):
    start_date = (datetime.today() - timedelta(days=7)).strftime("%m/%d/%Y")
    end_date = datetime.now().strftime("%m/%d/%Y")
    df = pd.DataFrame(index_history(symbol,start_date,end_date))
    dt = dt.strftime("%d %b %Y")
    filt = (df['HistoricalDate'] == dt)
    try:
        result = df.loc[filt,'HIGH'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def getlowofadate(symbol, dt):
    start_date = (datetime.today() - timedelta(days=7)).strftime("%m/%d/%Y")
    end_date = datetime.now().strftime("%m/%d/%Y")
    df = pd.DataFrame(index_history(symbol,start_date,end_date))
    dt = dt.strftime("%d %b %Y")
    filt = (df['HistoricalDate'] == dt)
    try:
        result = df.loc[filt,'LOW'].values[0]
        return float(result)
    except IndexError:
        return "NA"
def derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice):
    optionType = "CE"
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].min())
        return float(result)
    except:
        return "NA"
def derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,):
    optionType = "PE"
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].max())
        return float(result)
    except:
        return "NA"
def round_up_to_base(x, base=50):
    return x + (base - x) % base
def round_down_to_base(x, base=50):
    return x - (x % base)

In [7]:
# clear data 
wb = xw.Book('Formula R1 input data.xlsx') 
sheet = wb.sheets['R2']
sheet.range("C5:D6").clear_contents()
sheet.range("C11:C19").clear_contents()
sheet.range("E11:E20").clear_contents()
sheet.range("I11:I19").clear_contents()
sheet.range("K11:K20").clear_contents()

In [10]:
# Filling rows 4,5,6
symbol = "NIFTY 50"
start_date = sheet['C4'].value
end_date = sheet['D4'].value
expiry_date = sheet['J3'].value
instrumentType = "options"
highofstartdate = gethighofadate(symbol,sheet['C4'].value)
sheet['C5'].value= highofstartdate
lowofstartdate = getlowofadate(symbol,sheet['C4'].value)
sheet['C6'].value = lowofstartdate
highofenddate = gethighofadate(symbol,sheet['D4'].value)
sheet['D5'].value= highofenddate
lowofenddate = getlowofadate(symbol,sheet['D4'].value)
sheet['D6'].value = lowofenddate
maxof2days = max(highofstartdate,highofenddate)
sheet['E5'].value = maxof2days
minof2days = min(lowofstartdate,lowofenddate)
sheet['E6'].value = minof2days
bufferhigh = round(maxof2days * 1.0015)
sheet['F5'].value = bufferhigh
bufferlow = round(minof2days * 0.9985)
sheet['F6'].value = bufferlow
callendstrike = round_down_to_base(bufferlow)
sheet['K5'].value = callendstrike
putendstrike = round_up_to_base(bufferhigh)
sheet['K6'].value = putendstrike

In [14]:
# finding CallEntryStrike 1st row
symbol = "NIFTY"
start_date = sheet['C4'].value
end_date = sheet['D4'].value
expiry_date = sheet['J3'].value
instrumentType = "options"
strike = []
premium = []
twodll = []
strike.append(callendstrike)
temppremium = callendstrike * (0.85/100)
premium.append(temppremium)
temptwodll = derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[0])
twodll.append(temptwodll)

[21600]
[183.60000000000002]
[256.65]


In [23]:
# Call Entry strke 2....n rows
for i in range(1,9):
    if twodll[-1] >= premium[-1]:
        strike.append(strike[-1]+50)
        premium.append(strike[-1]* (0.85/100))
        twodll.append(derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[-1]))
    else:
        break
calldf = pd.DataFrame({"strike":strike,"premium":premium,"twodll":twodll})
calldf['diff']=round(calldf['twodll']-calldf['premium'],2)
print (calldf)
leastdiff = calldf.loc[calldf['diff']>0,'diff'].min()
leastdiff = round(leastdiff,2)
print (leastdiff)
    


   strike  premium  twodll   diff
0   21600  183.600  256.65  73.05
1   21650  184.025  226.25  42.22
2   21700  184.450  197.00  12.55
3   21750  184.875  171.40 -13.47
12.55
